In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [7]:
device = torch.cuda.current_device()
print(device)

0


In [6]:
torch.cuda.get_device_name(device)

'GeForce RTX 3080'

In [8]:
torch.cuda.memory_allocated()

0

In [10]:
torch.cuda.memory_reserved()

0

In [11]:
a = torch.FloatTensor([1.0,2.0])

In [12]:
a

tensor([1., 2.])

In [14]:
a.device

device(type='cpu')

In [15]:
a = torch.FloatTensor([1.0,2.0]).cuda()

In [16]:
a.device

device(type='cuda', index=0)

In [17]:
torch.cuda.memory_allocated()

512

In [18]:
torch.cuda.memory_reserved()

2097152

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)
        
        
    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.out(X)
        
        return X

In [24]:
torch.manual_seed(32)
model = Model()

In [26]:
next(model.parameters()).is_cuda

False

In [27]:
gpumodel = model.cuda()

In [28]:
next(gpumodel.parameters()).is_cuda

True

In [29]:
df = pd.read_csv("../Data/iris.csv")

In [31]:
X = df.drop('target', axis=1).values

In [32]:
y = df["target"].values

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=33)

In [36]:
X_train = torch.FloatTensor(X_train).cuda()

In [37]:
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.FloatTensor(y_train).cuda()
y_test = torch.FloatTensor(y_test).cuda()

In [38]:
X_train.device

device(type='cuda', index=0)

In [39]:
train_loader = DataLoader(X_train, batch_size=64, shuffle=True)
test_loader = DataLoader(X_test, batch_size=64, shuffle=False)